## DATASET: Market Basket
## Topic: Association Rule Mining
## Algo: Relim (Recursive Elimination)
### Goal: To measure and report 
- execution time & minimum support threshold
- number of frequent itemset
- representative associatino rules ( support, confidence, lift)
- observations on scalability and memory use

In [177]:
import pandas as pd
import time
import tracemalloc
from collections import Counter
import numpy as np

In [178]:
# step 1: Detect unique items and define the global item order by their frequencies
df = pd.read_csv('Groceries_dataset.csv')
item_counts = Counter(df['itemDescription']) # returns a dictionary: Counter({item: count})
unique_by_freq = [item[0] for item in item_counts.most_common()] #item_counts.most_common() returns a list of sorted tuples
len(unique_by_freq)


167

There are 167 unique items in the dataset, and I've sorted them by their frequencies. 

Next I will group them by member_number and date to form transactions.

In [179]:
transactions = (df.groupby(['Member_number','Date'])['itemDescription'].apply(list).tolist())
item_index = {item: idx for idx, item in enumerate(unique_by_freq)}
transactions = [sorted(t, key=lambda x: item_index[x]) for t in transactions] # within each transaction, items are also in global freq order
len(transactions)


14963

Together there are 14963 transactions, the original dataset has 38765 rows.

For min_support, I've got the item_counts as a dictionary which contain all the unique single item and its count, so I just need to iterate through the dict and get a average support of all the unique items, and I will set that to be my min_support. 

In [180]:
supports = np.array([count / len(transactions) for count in item_counts.values()])
minsup = np.percentile(supports, 50)   # top 50% of single-item supports
minsup

np.float64(0.005680679008220277)

The min_support I got is 0.0057 which matches the typical min_support range a 10k+ dataset size would have. 

Then I shall trim my unique single item to eliminate the less frequent items.

In [181]:
freq_item = [item for item, count in item_counts.items()if count/ len(transactions)>= minsup]

freq_item = sorted(freq_item, key= lambda x: item_counts[x], reverse=True)
len(freq_item)

84

NOW the Relim Algo Begins here, the core of Relim is that:
- picking a prefix item from my globally ordered (and trimmed frequent item list)
- creating conditional DB of what comes after that item,
- recursively mining those smaller DB

In [182]:
# Global total number of transactions
total = len(transactions)

def relim(transactions, minsup, prefix=[], results_dict={}, total=len(transactions)):
    # Initialize prefix and results container


    # Count all items appearing in the current conditional database
    counts = Counter(item for t in transactions for item in t)

    # Prune infrequent items (compare to global ratio)
    # Here c / total uses global total, not local len(condDB)
    freq_items = [i for i, c in counts.items() if c / total >= minsup]


    # Sort items by ascending frequency for recursion stability (Relim convention)
    freq_items.sort(key=lambda x: counts[x])

    # Explore each frequent item recursively
    for item in freq_items:
        new_prefix = prefix + [item]
        support = counts[item] / total  # global support ratio
        key = frozenset(new_prefix)

        # Record only multi-item sets (skip singletons, already known from preprocessing)
        # Deduplicate by keeping the highest support if re-encountered
        if len(new_prefix) > 1:
            if key not in results_dict or support > results_dict[key]:
                results_dict[key] = support

        # Build conditional database for this item
        condDB = []
        for t in transactions:
            if item in t:
                idx = t.index(item)
                suffix = t[idx + 1:]  # Only items after current one
                if suffix:
                    condDB.append(suffix)

        # Recurse into the conditional database with same global total
        relim(condDB, minsup, new_prefix, results_dict, total)

    # Return the dictionary of all discovered itemsets and supports
    return results_dict

In [183]:
freq_result = relim(transactions,minsup)
freq_result

{frozenset({'sausage', 'yogurt'}): 0.005948005079195348,
 frozenset({'soda', 'yogurt'}): 0.006014836596939116,
 frozenset({'sausage', 'soda'}): 0.006014836596939116,
 frozenset({'rolls/buns', 'root vegetables'}): 0.006081668114682885,
 frozenset({'rolls/buns', 'tropical fruit'}): 0.006081668114682885,
 frozenset({'rolls/buns', 'yogurt'}): 0.008153445164739691,
 frozenset({'rolls/buns', 'soda'}): 0.00848760275345853,
 frozenset({'other vegetables', 'sausage'}): 0.006348994185657956,
 frozenset({'other vegetables', 'tropical fruit'}): 0.006482657221145492,
 frozenset({'other vegetables', 'yogurt'}): 0.008353939717970995,
 frozenset({'other vegetables', 'soda'}): 0.01015839069705273,
 frozenset({'other vegetables', 'rolls/buns'}): 0.011227694980953017,
 frozenset({'canned beer', 'whole milk'}): 0.006081668114682885,
 frozenset({'pastry', 'whole milk'}): 0.0065494887388892606,
 frozenset({'shopping bags', 'whole milk'}): 0.006616320256633028,
 frozenset({'pip fruit', 'whole milk'}): 0.0067

In [184]:
print(f"Frequent itemset found in total: {len(freq_result)+len(freq_item)} including the singletons")
print(f"{len(freq_result)} excluding the singletons")

Frequent itemset found in total: 111 including the singletons
27 excluding the singletons


In [185]:
tracemalloc.start()
start_time = time.perf_counter()

relim(transactions, minsup)

end_time = time.perf_counter()
current, peak = tracemalloc.get_traced_memory()
tracemalloc.stop()

print(f"Runtime:{end_time - start_time :.4f} sec")
print(f"Peak memory: {peak/1024.1024 :.3f} MB")


Runtime:0.0833 sec
Peak memory: 216.740 MB


The most frequent itemset are:

 frozenset({'whole milk', 'yogurt'}): 0.011628684087415625,

 frozenset({'soda', 'whole milk'}): 0.012430662300340841,

 frozenset({'rolls/buns', 'whole milk'}): 0.014569270868141415,

 frozenset({'other vegetables', 'whole milk'}): 0.01537124908106663,
 
 frozenset({'whole milk', 'sausage'}): 0.009222749448639978,

In [186]:
# whole milk -> yogurt confidence = sup(whole milk , yogurt)/sup(whole milk)
def confidence_and_lift(a,b,itemset_support,total):
    # support ratios:
    sup_a = item_counts[a]/total
    sup_b = item_counts[b]/total


   #confidence ratios
    conf_a2b = itemset_support / sup_a
    conf_b2a = itemset_support / sup_b
    #lift - symmetric
    lift = itemset_support/ sup_a / sup_b

    print(f"({a} → {b})  | confidence: {conf_a2b:.3f}")
    print(f"({b} → {a})  | confidence: {conf_b2a:.3f}")
    print(f"Lift: {lift:.3f}")

In [187]:
confidence_and_lift('whole milk','sausage',0.009222749448639978,len(transactions))

(whole milk → sausage)  | confidence: 0.055
(sausage → whole milk)  | confidence: 0.149
Lift: 0.893


In [188]:
confidence_and_lift('other vegetables', 'whole milk', 0.01537124908106663,len(transactions))

(other vegetables → whole milk)  | confidence: 0.121
(whole milk → other vegetables)  | confidence: 0.092
Lift: 0.725


In [189]:
confidence_and_lift('rolls/buns', 'whole milk', 0.014569270868141415,len(transactions))

(rolls/buns → whole milk)  | confidence: 0.127
(whole milk → rolls/buns)  | confidence: 0.087
Lift: 0.760


In [190]:
confidence_and_lift('soda', 'whole milk',0.012430662300340841,len(transactions))

(soda → whole milk)  | confidence: 0.123
(whole milk → soda)  | confidence: 0.074
Lift: 0.735


In [191]:
confidence_and_lift('whole milk', 'yogurt', 0.011628684087415625,len(transactions))

(whole milk → yogurt)  | confidence: 0.070
(yogurt → whole milk)  | confidence: 0.130
Lift: 0.780
